In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np
import math

Загружаем датафрейм и модифицируем его для дальнейшей работы

In [ ]:
df = pd.read_excel('dataset.xlsx', sheet_name='1')
df = df.drop(columns='Unnamed: 0')
df.columns = df.iloc[0]
df = df.drop(index=0)
df.index = df.index - 1
df = df.rename(columns={'curs (rub/USD)' : 'curs'})

,data,curs
0,2022-08-26 00:00:00,59.7699
1,2022-08-25 00:00:00,59.9974
2,2022-08-24 00:00:00,59.8963
3,2022-08-23 00:00:00,59.7419
4,2022-08-20 00:00:00,59.1321
...,...,...
400,2021-01-15 00:00:00,73.7961
401,2021-01-14 00:00:00,73.5264
402,2021-01-13 00:00:00,74.2663
403,2021-01-12 00:00:00,74.5157


Покажем на конретном примере как считается $CVaR(p)$ (здесь $p = 5%$)

In [ ]:
df['returns'] = df.curs/(df.curs.shift(1))
ret = df.returns
ret = ret.dropna()
ret = ret.astype('float')
ret = pd.DataFrame(ret.sort_values(axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last'))
ret = ret.reset_index().drop(columns='index')
p = 0.05
count = len(ret) # 404
position = round(p*count) + 1
cvar_p = np.mean(ranked_returns[0 : position]) # take average

19


returns   -0.018642
Name: 19, dtype: float64

Чтобы каждый раз не менять $p$ и данные напишем функцию, которая $CVaR(p)$ для любых данных и любого $p$ (в процентах).

In [ ]:
def ln_cvar(data : pd.Series, p : float) -> float:
    returns = data/data.shift(1)
    ranked_returns =  pd.DataFrame(returns.sort_values(axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last'))
    ranked_returns = ranked_returns.dropna()
    count = len(ranked_returns)
    position = round(p*count) + 1
    cvar_p = np.mean(ranked_returns.iloc[0 : position])
    return cvar_p[0]

Проверка, что функция посчитала всё правильно

In [ ]:
print(ln_cvar(df.curs, 0.05))

-0.018641883817865745
-0.05733805903848952


c:\Users\Валерия\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
